In [19]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
base = "https://de.wikipedi0.org"

def format_page(page):
    '''formats the page so it can be used for scraping. uses requests and beautifulsoup'''
    page_ready = requests.get(page)
    page_ready = BeautifulSoup(page_ready.text, "html.parser").body
    return page_ready

def prepare_bad_urls(base):
    '''returns list of wikipedia subsites which shouldn't be used for the game. takes full wikipedia link as a base, e.g. base = "https://de.wikipedi0.org"'''
    bad = ["/w/", "/Wikipedia", "/wiki/Hilfe", "/wiki/Spezial", "/wiki/Datei", "/wiki/Kategorie", "/wiki/Portal", "/wiki/Wikipedia:", "/wiki/Diskussion:"]
    full_bad_urls = []
    for bad_url in bad:
        full_bad_url = urllib.parse.urljoin(base, bad_url)
        full_bad_urls.append(full_bad_url)
    return full_bad_urls

def return_wiki_links(page):
    '''returns wiki links in a page. needs format_page and filter_bad_urls functions.'''
    full_bad_urls = prepare_bad_urls(base)
    wiki_links_in_page = []
    page_text = format_page(page)
    links_in_page = page_text.find_all('a', href=True)
    
    for link in links_in_page:
        full_url = urllib.parse.urljoin(base, link['href'])
        
        if "#" in full_url:
            full_url = full_url[:full_url.find("#")]
        
        if full_url == base:
            continue
        
        if not full_url.startswith(base):
            continue 
            
        is_bad_url = False
        for fbu in full_bad_urls:
            if full_url.startswith(fbu):
                is_bad_url = True
                break
        if is_bad_url == True:
            continue
        
        wiki_links_in_page.append(full_url)
        
    return list(set(wiki_links_in_page))

In [26]:
import copy
import itertools

def wikipedia_game(start_page, goal_page):
    
    if goal_page == start_page:
        return("RESULT: no need for clicking. You are already there: The two pages are identical.")

    round_count = 1
    already_tried = [start_page]
    links_to_test = return_wiki_links(start_page)
    print(goal_page)

    while True:
        if round_count > 1:
            links_to_test = copy.deepcopy(new_links)
        else:
            new_links = []
        
        if goal_page in links_to_test:
            return print("RESULT: The goal page %s can be reached with %s clicks: %s connects %s with %s" %(goal_page, str(round_count), start_page, start_page, goal_page))
        else:
            print("no results in round %s. starting round %s" %(round_count, (round_count + 1)))
            round_count += 1

        for link in links_to_test: #liste mit allen links erstellen, die für diese runde relevant sind
            new_link_bunch = return_wiki_links(link)
            new_links_bunch = [x for x in new_link_bunch if x not in already_tried]
            
            if goal_page in new_links_bunch:
                return print("RESULT: The goal page %s can be reached with %s clicks: %s connects %s with %s" %(goal_page, str(round_count), link, start_page, goal_page))
                # was hier noch fehlt: funktion, die zurückverfolgt, welche links wohin führen! 
            new_links.append(new_links_bunch)

        already_tried.append(new_links)
        new_links = list(set(list(itertools.chain.from_iterable(new_links))))

blub = wikipedia_game("https://de.wikipedi0.org/wiki/Augenblickswert", 'https://de.wikipedi0.org/wiki/Adolf_Hitler')
# a = wikipedia_game('https://de.wikipedi0.org/wiki/Augenblickswert', 'https://de.wikipedi0.org/wiki/Adolf_Hitler')

https://de.wikipedi0.org/wiki/Adolf_Hitler
no results in round 1. starting round 2
no results in round 2
no results in round 2. starting round 3
no results in round 3
no results in round 3. starting round 4
RESULT: The goal page https://de.wikipedi0.org/wiki/Adolf_Hitler can be reached with 4 clicks: https://de.wikipedi0.org/wiki/Rinteln connects https://de.wikipedi0.org/wiki/Augenblickswert with https://de.wikipedi0.org/wiki/Adolf_Hitler
